        {% raw %}

# `jekyll` & `jupyter`
---

---

__<del>Two</del>__ templating steps are necessary to use `jekyll` a blogging engine for `notebook`s 

1. convert a __notebook__ to markdown with `nbconvert` and `jinja2` templates
2. `jekyll` will compile `liquid` before creating the static `_site`.

> In this `post`, we establish the conversion pipeline to blog notebooks. 

## Configuring `nbconvert`

`nbconvert` transforms `ipynb` files to other formats; the configuration system provides flexibility the conversion.  

In [66]:
        %%file ../_layouts/markdown.py
        c.ExtractOutputPreprocessor.enabled=False

Overwriting ../_layouts/markdown.py


## 1. the `nbconvert` template

This template converts a notebook to `markdown` that may be ingested by `jekyll`.  We will call this template `jekyll.md.tpl` and place it in `_layouts`, the preferred directory for `jekyll` layouts.  `jekyll.md.tpl` extends the current [`nbconvert.templates.markdown`](https://github.com/jupyter/nbconvert/blob/master/nbconvert/templates/markdown.tpl) template

In [71]:
        %%file ../_layouts/jekyll.md.tpl
        {% extends 'markdown.tpl' %}

Overwriting ../_layouts/jekyll.md.tpl


### frontmatter

`jekyll` recognizes files with `--- front matter ---`.  `nbconvert` will transform markdownify the notebook and preprend the front matter.  The transformed file will be used by the `jekyll` server to create a static site.

> This template adds the notebook metadata to the front matter.

In [72]:
        %%file ../_layouts/jekyll.md.tpl -a
        {%- block header scoped-%}
        ---
        layout: post
        {{ resources.metadata | dump | load | yaml(default_flow_style=False)}}
        ---
        {{super()}}
        {% endblock header %}

Appending to ../_layouts/jekyll.md.tpl


In [66]:
        %%file ../_layouts/markdown.py -a
        c.TemplateExporter.filters = {
            'dump': 'json.dumps',
            'load': 'json.loads',
            'yaml': 'yaml.dump'}

Overwriting ../_layouts/markdown.py


### javascript

It is necessary to place `javascript` into the markdown files; I commonly use `bokeh` and `coffeescript` in notebook.  The `data_javascript` block is not defined for the `nbconvert.template.markdown`, but it is in `nbconvert.template.html.basic`.  With some [copy🍝](https://github.com/jupyter/nbconvert/blob/master/nbconvert/templates/html/basic.tpl#L212) we add `javascript` this to the template

In [73]:
        %%file ../_layouts/jekyll.md.tpl -a
        {%- block data_javascript scoped %}
        {% set div_id = uuid4() %}
        <div id="{{ div_id }}"></div>
        <div class="output_subarea output_javascript {{ extra_class }}">
        <script type="text/javascript">
        var element = $('#{{ div_id }}');
        {{ output.data['application/javascript'] }}
        </script>
        </div>
        {%- endblock -%}

Appending to ../_layouts/jekyll.md.tpl


In [66]:
        %%file ../_layouts/markdown.py -a
        c.NbConvertBase.display_data_priority = ['text/html', 'text/markdown', 'image/svg+xml', 'text/latex', 'image/png', 'image/jpeg', 'text/plain', 'application/javascript']

Overwriting ../_layouts/markdown.py


### `stream` output

Standard python output - `stream` - should be notably separate from inputs.  Add horizontal rules to separate the `stream` output

In [74]:
        %%file ../_layouts/jekyll.md.tpl -a
        {% block stream %}
        ---
        {{ output.text | indent }}
        ---
        {% endblock stream %}
        

Appending to ../_layouts/jekyll.md.tpl


## `nbconvert` configuration



## the `jekyll` `_layout`

In [64]:
        post = '2015-12-27-template.ipynb'

In [65]:
        from plumbum import local, FG
        from bs4 import BeautifulSoup
        import nbformat, nbconvert

In [ ]:
        body = nbconvert.export(
            nbconvert.get_exporter('html'), nbformat.read(post, 4))[0]
        soup = BeautifulSoup(body, 'lxml')
        div = soup.new_tag('div', id='notebook-container', **{'class': "container"})        

### customization

In [96]:
        div.append('{{content}}')
        div.append('{% include disqus.html %}')
        soup.select_one('#notebook-container').replace_with(div)
        soup.select_one('#notebook-container').insert_before('{% include header.html %}')
        pass

In [97]:
#         with open('../_layouts/post.html', 'w') as f:
#             f.write(str(soup))

        {% endraw %}